**Установка библиотек**

In [1]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 9.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=fd26e8f6db0bfa16c99df777080a2b7db8e57bdf958e1f399d1927371f8a5b99
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


**Импорт библиотек**

In [2]:
import pandas as pd
from langdetect import detect
import re
from collections import Counter

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Обработка данных**

In [5]:
datadir = '/content/drive/MyDrive/Colab Notebooks/pet_projects/abstract_sum/results/' + 'vkr.csv'
df = pd.read_csv(datadir)
print('Изначальное кол-во строк: ', df.shape[0])
df = df.dropna(subset=['abstract'])
print('Кол-во строк после удаления Нанов: ', df.shape[0])

Изначальное кол-во строк:  72830
Кол-во строк после удаления Нанов:  68976


In [6]:
df.head(2)

,vkrurl,abstract,has_file,hasEnVersion,id,learnProgram_id,learnProgram_title,level,orgUnit,rating,student,title,year
0,https://www.hse.ru/edu/vkr/206738445,В данной дипломной работе описывается разработ...,False,True,206738445,135181640.0,Информатика и вычислительная техника,Бакалавриат,Московский институт электроники и математики и...,NaN,Зверев Валерий Дмитриевич,Разработка веб-приложения для содания модели у...,2017
1,https://www.hse.ru/edu/vkr/153010267,Выбор между активным и пассивным управлением я...,False,False,153010267,135181656.0,Экономика,Бакалавриат,Факультет экономических наук,NaN,Гильманова Диана Маратовна,Оценка эффективности активно и пассивно управл...,2015


In [7]:
# re: удаляем тэги, табы, переносы строк, новые строки
abstracts = df['abstract']
clean_abstracts = []
cleanr1 = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') # tags
cleanr2 = re.compile('\s+') # tabs, newlines, spaces
cleanr3 = re.compile(' +') # consecutive spaces
df['abstract'] = df['abstract'].str.replace(cleanr1, '')
df['abstract'] = df['abstract'].str.replace(cleanr2, ' ')
df['abstract'] = df['abstract'].str.replace(cleanr3, ' ')
df['title_length'] = df['title'].str.len()
df['title_wordcnt'] = df['title'].str.split().apply(len)
df['abstract_length'] = df['abstract'].str.len()
df['abstract_wordcnt'] = df['abstract'].str.split().apply(len)
df = df[df['abstract_wordcnt'] > 30]
print('Кол-во строк после удаления слишком коротких абстрактов: ', df.shape[0])

Кол-во строк после удаления слишком коротких абстрактов:  65412


In [8]:
# langdetect: размечаем язык абстракта
df['lang'] = df['abstract'].apply(detect)

In [9]:
Counter(df['lang'])

Counter({'en': 551, 'fr': 1, 'ru': 64860})

In [10]:
df = df[df['lang'] == 'ru']
print('Кол-во строк после фильтрации по языку: ', df.shape[0])

Кол-во строк после фильтрации по языку:  64860


In [11]:
cols = ['vkrurl', 'title', 'abstract', 'title_wordcnt', 'abstract_wordcnt', 
        'title_length', 'abstract_length', 'level', 'learnProgram_title', 
        'rating', 'year']
dfpubl = df[cols]

In [12]:
resultdir = '/content/drive/MyDrive/Colab Notebooks/pet_projects/abstract_sum/data/' + 'cleanvkr.csv'
dfpubl.to_csv(resultdir, index = False)
pd.read_csv(resultdir).head(2)

,vkrurl,title,abstract,title_wordcnt,abstract_wordcnt,title_length,abstract_length,level,learnProgram_title,rating,year
0,https://www.hse.ru/edu/vkr/206738445,Разработка веб-приложения для содания модели у...,В данной дипломной работе описывается разработ...,10,81,76,629,Бакалавриат,Информатика и вычислительная техника,NaN,2017
1,https://www.hse.ru/edu/vkr/153010267,Оценка эффективности активно и пассивно управл...,Выбор между активным и пассивным управлением я...,12,103,100,839,Бакалавриат,Экономика,NaN,2015
